# MVSep-MDX23 Colab Fork v2.3
Адаптация алгоритма MVSep-MDX23 для Google Colab, с различными надстройками:

https://colab.research.google.com/github/alphatoasterous/MVSEP-MDX23-Colab_v2/blob/v2.3/MVSep-MDX23-Colab.ipynb

Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Модели: [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Адаптация и надстройки: [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
* Перевод на русский и интерактивная загрузка: [alphatoaster](https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2/)
</font>

In [ ]:
#@markdown #Установка
#@markdown *Запустите эту ячейку для установки MVSep-MDX23*
print('Установка... Это займёт около 1 минуты...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2.git &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2
!pip install -r requirements.txt &> /dev/null
# onnxruntime-gpu nightly fix for cuda12.2
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
print('Installation done !')

### О параметрах:

Стандартная конфигурация оптимально работает на бесплатном T4 инстансе Google Colab, можете включить VOCFT для *возможно* результата получше.
 Двигайте ползунки на свой страх и риск!
<font size=2>

* **BigShifts :** Качество/скорость лучше при значениях от 3 до 11, **НО** 11 не всегда дает наилучшие результаты. Работает примерно как seed: разные значения дают разные результаты.<br>
Чем больше значение, тем дольше обработка.
</font>

<font size=2>

* **Overlap InstVoc/VitLarge :** Нет преимущества в использовании высоких значений, если BigShifts на дефолтном значении. Если BigShifts=1 (обычная обработка), используйте более высокие значения, например 8 или даже 16.<br>
Чем больше значение, тем дольше обработка.<br>
 *То же самое относится и к overlap_VOCFT, но со значениями от 0 до 0.95.*
</font>

<font size=2>

* **Use VOCFT :** Определяет, использовать VOCFT или нет. <br>
Обработка дольше, качество *возможно* выше<br>
</font>

<font size=2>

* **Weights :** Взвешивание моделей: насколько приоритетнее результат обработки данной модели для вырезаемого трека.
</font>


In [ ]:
#@markdown #Загрузка аудио
#@markdown *Запустите эту ячейку для загрузки аудиофайлов (WAV, FLAC, MP3 или zip-файлов, содержащих аудиофайлы) в этот Colab.*

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Задать директорию вывода, создать если её нет
target_dir = '/content/audio_files/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Если файл - zip-архив
    if fn.endswith('.zip'):
        # Записать архив
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Распаковать архив
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Удалить архив
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip-архив "{name}" разархивирован. Файлы находятся в: {folder}'.format(name=fn, folder=unzip_dir))

        # Вывести кнопку ссылки для каждого файла
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Разархивирован файл "{name}" с размером {length} байт'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='Файл сохранен: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # Если файл - не zip-архив
    # Сохранить файл в директорию
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='Пользователь загрузил файл "{name}" с размером {length} байт'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Проверить, если файл поддерживаемого формата: .wav, .mp3, .flac
    if not (fn.endswith('.wav') or fn.endswith('.mp3') or fn.endswith('.flac')):
        warning_text = widgets.HTML(
            value='<b style="color: red;">ВНИМАНИЕ:</b> Вы загружаете неподдерживаемый файл.'
        )
        display(warning_text)

    # Создать кликабельную ссылку на файл
    file_path_text = widgets.HTML(
        value='Файл сохранен: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown #Разделение трека
#@markdown Скопируйте путь, полученный в ячейке загрузки, если аудио было загружено через неё
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2


#@markdown input = /content/audio_files для загруженных через ячейку загрузки аудиофайлов
input = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
#@markdown ---
#@markdown *Bigshifts=1 чтобы отключить BigShifts*

BigShifts = 7 #@param {type:"slider", min:1, max:41, step:1}
#@markdown ---
overlap_InstVoc = 1 #@param {type:"slider", min:1, max:40, step:1}
overlap_VitLarge = 1 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---
weight_InstVoc = 8 #@param {type:"slider", min:0, max:10, step:1}
weight_VitLarge = 5 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_VOCFT = False #@param {type:"boolean"}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
vocals_instru_only = True #@param {type:"boolean"}
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
output_format = 'PCM_16' #@param ["PCM_16", "FLOAT"]
if vocals_instru_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''


if use_VOCFT:
    use_VOCFT = '--use_VOCFT true'
else:
    use_VOCFT = ''

if Path(input).is_file():
  file_path = input
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --large_gpu \
        --weight_InstVoc {weight_InstVoc} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstVoc {overlap_InstVoc} \
        --overlap_VitLarge {overlap_VitLarge} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        {vocals_only} \
        {use_VOCFT}

else:
  file_paths = sorted([f'"{glob.escape(path)}"' for path in glob.glob(input + "/*")])[:]
  input_audio_args = ' '.join(file_paths)
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
          --large_gpu \
          --weight_InstVoc {weight_InstVoc} \
          --weight_VOCFT {weight_VOCFT} \
          --weight_VitLarge {weight_VitLarge} \
          --input_audio {input_audio_args} \
          --overlap_demucs {overlap_demucs} \
          --overlap_VOCFT {overlap_VOCFT} \
          --overlap_InstVoc {int(overlap_InstVoc)} \
          --overlap_VitLarge {int(overlap_VitLarge)} \
          --output_format {output_format} \
          --BigShifts {BigShifts} \
          --output_folder "{output_folder}" \
          {vocals_only} \
          {use_VOCFT}
